In [12]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
import hashlib
import sympy as sp


euler_value = sp.E


Encrypted = bytes.fromhex('805534c14e694348a67da0d75165623cf603c2a98405b34fe3ba8752ce24f5040c39873ec2150a61591b233490449b8b7bedaf83aa9d4b57d6469cd3f78fdf55')
iv = Encrypted[0:16]
encrypt = Encrypted[16:]
def decrypt(encrypted, key):
    encrypted_bytes = encrypted
    iv = encrypted_bytes[:16]
    ciphertext = encrypted_bytes[16:]
    
    
    
    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted_message = cipher.decrypt(ciphertext)
    
    return decrypted_message

x = sp.Symbol('x')


array = [[0, 14], [14, 0], [2, 12], [12, 2], [2, 14], [14, 2], [4, 10], [10, 4], [4, 14], [14, 4], [6, 8], [8, 6], [6, 10], [10, 6], [6, 12], [12, 6], [6, 14], [14, 6], [8, 6], [6, 8], [8, 14], [14, 8], [10, 4], [4, 10], [10, 6], [6, 10], [10, 12], [12, 10], [10, 14], [14, 10], [12, 2], [2, 12], [12, 6], [6, 12], [12, 10], [10, 12], [12, 14], [14, 12], [14, 0], [0, 14], [14, 2], [2, 14], [14, 4], [4, 14], [14, 6], [6, 14], [14, 8], [8, 14], [14, 10], [10, 14], [14, 12], [12, 14], [14, 14], [14, 14]]


key = 60


for val in array:
    c1 = val[0]
    c2 = val[1]
    C1, C2 = sp.symbols('C1 C2')
    f = C1 * euler_value ** (((3 + sp.sqrt(3)) / 2) * x) + C2 * euler_value ** (((3 - sp.sqrt(3)) / 2) * x)
    f_prime = sp.diff(f, x)
    
    
    f_second_prime = sp.diff(f_prime, x)
    init_cond_1 = f.subs(x, 0) - c1
    init_cond_2 = f_prime.subs(x, 0)- c2
    sol = sp.solve([init_cond_1, init_cond_2], (C1, C2)) 
    
    f = f.subs(sol)
  
   
    point = f.subs(x, key).evalf(100)
    
    point_hash = hashlib.sha256(str(point).encode()).digest()[:16]
   
    
    decrypted_message = decrypt(Encrypted,point_hash)
    try :
        print(decrypted_message.decode())
        exit()
    except :
        pass

AKASEC{d1d_y0u_3nj0y_c41cu1u5_101?}
AKASEC{d1d_y0u_3nj0y_c41cu1u5_101?}


: 

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import hashlib
import sympy as sp
array = []
for x in range(100) :
    for y in range(100) :
        if  x | y == 14 :
            array.append([x,y])
            array.append([y,x])
for x in array :
    assert (x[0] | x[1]) == 14
print(array)

In [ ]:
from sage.all import *
import hashlib
import sympy as sp
from Crypto.Cipher import AES

key = 60
encrypted = bytes.fromhex("805534c14e694348a67da0d75165623cf603c2a98405b34fe3ba8752ce24f5040c39873ec2150a61591b233490449b8b7bedaf83aa9d4b57d6469cd3f78fdf55")
iv = encrypted[:16]
ct = encrypted[16:]

for v0 in range(2**4):
    for v1 in range(2**4):
        if v0 | v1 != 14:
            continue

        # Set up and solve the differential equation
        x = var('x')
        y = function('y')(x)
        yp = diff(y, x)
        ypp = diff(yp, x)
        f = desolve(2*ypp - 6*yp + 3*y == 0, y, ics=[0,v0,v1], ivar=x)

        fp = derivative(f, x)
        assert int(fp(x=0)) == v1 and int(f(x=0)) == v0
        
        # Test for flag
        point = f(x=key)
        # point = str(point).replace('^', '**') 
        # # The challenge is flawed here, to get the flag we need sympy to derive the wrong/incorrect answer as it mistakes sage's exponentiation for xor.

        val = sp.N(point, 100)
        point_hash = hashlib.sha256(str(val).encode()).digest()[:16]
        cipher = AES.new(key=point_hash, iv=iv, mode=AES.MODE_CBC)
        msg = cipher.decrypt(ct)
        if b'AKA' in msg:
            print(msg) # b'AKASEC{d1d_y0u_3nj0y_c41cu1u5_101?}\r\r\r\r\r\r\r\r\r\r\r\r\r'